In [1]:
import numpy as np 
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [2]:
def select_train(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

def select_test(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

In [3]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()

# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset1 = raw_dataset.copy()
test_dataset1.isna().sum()
test_dataset1 = test_dataset1.dropna()

# 2/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim2.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset2 = raw_dataset.copy()
test_dataset2.isna().sum()
test_dataset2 = test_dataset2.dropna()

# 3/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim3.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset3 = raw_dataset.copy()
test_dataset3.isna().sum()
test_dataset3 = test_dataset3.dropna()

In [4]:
list_comp = ['input_width', 'input_size','nb_params_conv', 'nb_params_bn', 'nb_params_fc', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']

list_var = ['input_width', 'nb_params_bn', 'nb_params_fc','nb_conv_layers', 'nb_bn_layers', 'nb_fc_layers']

train_dataset = select_train(list_var, train_dataset)
test_dataset1 = select_test(list_var, test_dataset1)
test_dataset2 = select_test(list_var, test_dataset2)
test_dataset3 = select_test(list_var, test_dataset3)

In [5]:
y_train = train_dataset.pop('ACET')
X_train = train_dataset
X_train.tail()

,Model_name,input_shape,input_size,nb_params_conv,flops,nb_layers,sum_activations,weighted_sum_neurons
1606,basic_model_700_7,"(1, 700, 700, 3)",1470000,1758116,196248875920,16,119497648,98010896800
1607,basic_model_320_23,"(1, 320, 320, 3)",307200,2499430,168678951282,44,298643104,84243523392
1608,basic_model_56_23,"(1, 56, 56, 3)",9408,28069064,170036087884,43,20701712,84982551552
1609,basic_model_224_33,"(1, 224, 224, 3)",150528,22242728,132106973640,60,31316236,66016370176
1610,basic_model_600_38,"(600, 600, 3)",1080000,23154824,274420498147,45,148258606,137108348476


In [6]:
Y_e1 = test_dataset1.pop('ACET')
X_e1 = test_dataset1

Y_e2 = test_dataset2.pop('ACET')
X_e2 = test_dataset2

Y_e3 = test_dataset3.pop('ACET')
X_e3 = test_dataset3

In [7]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

X_test_model_names1 = X_e1.pop('Model_name')
X_test_input_shapes1 = X_e1.pop('input_shape')

X_test_model_names2 = X_e2.pop('Model_name')
X_test_input_shapes2 = X_e2.pop('input_shape')

X_test_model_names3 = X_e3.pop('Model_name')
X_test_input_shapes3 = X_e3.pop('input_shape')

In [8]:
y_test1 = Y_e1.values
y_test2 = Y_e2.values
y_test3 = Y_e3.values

In [9]:
steps = [('scaler', StandardScaler()), ('RFR', RandomForestRegressor())]
pipeline = Pipeline(steps)

In [10]:
# gridSearchCV hyperparameters space
"""
param_grid = [
        {'RFR__bootstrap': [True,False],
         'RFR__max_depth': [None,10,20,30,40,50,60,70,80,90,100,120,140,160,180,200,220,240,260,280,300],
         'RFR__n_estimators': [100,200,300,400,500,600,700,800,900,1000],
         'RFR__max_features': ['auto', 'sqrt', 'log2', None],
         'RFR__min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9, 10],
         'RFR__min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
         'RFR__random_state': [42]
        },
    ]
"""

# optimal hyperparameters
param_grid = [
        {'RFR__bootstrap': [True],
         'RFR__max_depth': [20],
         'RFR__n_estimators': [200],
         'RFR__max_features': [None],
         'RFR__min_samples_leaf': [7],
         'RFR__min_samples_split': [3],
         'RFR__random_state': [42], 
        },
    ]

In [11]:
grid = GridSearchCV(pipeline, param_grid=param_grid, cv=2)

In [12]:
grid.fit(X_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('RFR', RandomForestRegressor())]),
             param_grid=[{'RFR__bootstrap': [True], 'RFR__max_depth': [20],
                          'RFR__max_features': [None],
                          'RFR__min_samples_leaf': [7],
                          'RFR__min_samples_split': [3],
                          'RFR__n_estimators': [200],
                          'RFR__random_state': [42]}])

In [13]:
print(grid.best_params_)

{'RFR__bootstrap': True, 'RFR__max_depth': 20, 'RFR__max_features': None, 'RFR__min_samples_leaf': 7, 'RFR__min_samples_split': 3, 'RFR__n_estimators': 200, 'RFR__random_state': 42}


In [14]:
best_pipe = grid.best_estimator_
best_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('RFR',
                 RandomForestRegressor(max_depth=20, max_features=None,
                                       min_samples_leaf=7, min_samples_split=3,
                                       n_estimators=200, random_state=42))])

In [15]:
y_pred1 = best_pipe.predict(X_e1.values)
y_pred2 = best_pipe.predict(X_e2.values)
y_pred3 = best_pipe.predict(X_e3.values)

In [24]:
y_test = y_test1
y_pred = y_pred1
test_models = X_test_model_names1.values
test_input_shapes = X_test_input_shapes1.values

In [25]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.9861625010699666
R2_value :  0.9831690135128095
Mean absolute error MAE : 8.46994365274252
Root Mean squared error RMSE : 13.572764537291745
Median absolute error MAD: 5.282800732592818


In [26]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE :', max(mape))
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE : 48.165468464084995
MAPE: 11.94 %.
Accuracy: 88.06 %.


In [27]:
import csv
with open('./measured_vs_predicted_RF_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'real_exec_time', 'predicted_exec_time', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])